# Convolutional Neural Network

Use this page to get a full understand.
https://cuijiahua.com/blog/2018/12/dl-10.html

## Load Dependent Packages

In [1]:
import torch
import torchvision

## Device configuration: CPU or GPU

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Select Hyper Parameters

In [3]:
num_epochs = 5
num_classes = 10
batch_size = 100 # Total 100 batches
learning_rate = 0.01

## Define Dataset Class

In [4]:
import numpy as np
import gzip
import os
class MNISTDataset(torch.utils.data.Dataset):
    def __init__(self, root, train=True, transform=None):
        # The file name prefix is obtained according to whether it is a training set or not.
        self.file_pre = 'train' if train == True else 't10k'
        self.transform = transform

        # Generate the image and label file path of the corresponding dataset.
        self.label_path = os.path.join(root, '%s-labels-idx1-ubyte.gz' % self.file_pre)
        self.image_path = os.path.join(root, '%s-images-idx3-ubyte.gz' % self.file_pre)

        # Read file data and return pictures and labels.
        self.images, self.labels = self.__read_data__(self.image_path, self.label_path)

    def __read_data__(self, image_path, label_path):
        # Data set reading.
        with gzip.open(label_path, 'rb') as lbpath:
            labels = np.frombuffer(lbpath.read(), np.uint8, offset=8)
        with gzip.open(image_path, 'rb') as imgpath:
            images = np.frombuffer(imgpath.read(), np.uint8, offset=16).reshape(len(labels), 28, 28)
        return images, labels

    def __getitem__(self, index):
        image, label = self.images[index], int(self.labels[index])
        
        # If you need to convert to tensor, use tansform.
        if self.transform is not None:
            image = self.transform(np.array(image))  # Avoid bug: use np.array
        return image, label

    def __len__(self):
        return len(self.labels)

Download Dataset from Online:
```python
train_dataset = torchvision.datasets.MNIST(root='../data/MNIST', train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='../data/MNIST', train=False, transform=torchvision.transfroms.ToTensor(), download=True)
```

## Load through Local Dataset

In [5]:
# If datasets have been downloaded already!
train_dataset = MNISTDataset('../data/MNIST/', transform=torchvision.transforms.ToTensor())
test_dataset = MNISTDataset('../data/MNIST/', train=False, transform=torchvision.transforms.ToTensor())

## Set Data loader (Input Pipeline)

In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Calculate the Input Parameters of the First Full Connection Layer:
```python
def fc_in(image, Conv, Pool):
    for i, j in zip(Conv, Pool):
        hk = (image[0] - i[0] + 2 * i[2]) / i[1] + 1
        wk = (image[1] - i[0] + 2 * i[2]) / i[1] + 1
        hp = (hk - j[0] + 2 * j[2]) / j[1] + 1
        wp = (wk - j[0] + 2 * j[2]) / j[1] + 1
        image = (hp, wp)
    return (int(image[0]), int(image[1]))
fc_in((28, 28), ((5, 1, 2), (5, 1, 2)), ((2, 2, 0), (2, 2, 0)))
```

## Define Convolutional Neural Network (Two Convolutional Layers)

In [7]:
class ConvNet(torch.nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
                                          torch.nn.BatchNorm2d(16),
                                          torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
                                          torch.nn.BatchNorm2d(32),
                                          torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = torch.nn.Linear(7 * 7 * 32, num_classes)
     
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        # out.size = (batchsize, channels, x, y)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

## Make Model with GPU

In [8]:
model = ConvNet(num_classes).to(device)

## Loss and Optimizer

In [9]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Train Model with GPU

In [10]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1519
Epoch [1/5], Step [200/600], Loss: 0.0671
Epoch [1/5], Step [300/600], Loss: 0.1410
Epoch [1/5], Step [400/600], Loss: 0.1034
Epoch [1/5], Step [500/600], Loss: 0.0701
Epoch [1/5], Step [600/600], Loss: 0.0220
Epoch [2/5], Step [100/600], Loss: 0.0092
Epoch [2/5], Step [200/600], Loss: 0.0312
Epoch [2/5], Step [300/600], Loss: 0.1994
Epoch [2/5], Step [400/600], Loss: 0.0334
Epoch [2/5], Step [500/600], Loss: 0.0097
Epoch [2/5], Step [600/600], Loss: 0.0768
Epoch [3/5], Step [100/600], Loss: 0.0480
Epoch [3/5], Step [200/600], Loss: 0.0338
Epoch [3/5], Step [300/600], Loss: 0.0429
Epoch [3/5], Step [400/600], Loss: 0.0221
Epoch [3/5], Step [500/600], Loss: 0.0179
Epoch [3/5], Step [600/600], Loss: 0.0190
Epoch [4/5], Step [100/600], Loss: 0.0191
Epoch [4/5], Step [200/600], Loss: 0.0046
Epoch [4/5], Step [300/600], Loss: 0.0341
Epoch [4/5], Step [400/600], Loss: 0.0213
Epoch [4/5], Step [500/600], Loss: 0.0509
Epoch [4/5], Step [600/600], Loss:

## Test Model with GPU

In [11]:
# Do not enable batchnormalization and dropout to ensure that BN and dropout do not change. The pytorch framework will automatically fix BN and dropout without averaging, but use the trained value.
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.56 %


## Save the Model Checkpoint

In [12]:
torch.save(model.state_dict(), 'cnn.ckpt')